In [1]:
%%writefile Dockerfile

FROM python:3.10-slim-buster
COPY requirements.txt .
RUN pip3 install -r requirements.txt
ENV PYTHONUNBUFFERED=TRUE
ENTRYPOINT ["python3"]

Overwriting Dockerfile


In [2]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'preprocessing-container'
tag = ':latest'
region = boto3.session.Session().region_name

In [3]:
%cd ~/SageMaker/aws-healthcare-lifescience-ai-ml-sample-notebooks/workshops/Caristo/1. Preprocessing

/home/ec2-user/SageMaker/aws-healthcare-lifescience-ai-ml-sample-notebooks/workshops/Caristo/1. Preprocessing


In [4]:
! pwd

/home/ec2-user/SageMaker/aws-healthcare-lifescience-ai-ml-sample-notebooks/workshops/Caristo/1. Preprocessing


In [5]:
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [8]:
! docker build -t $ecr_repository . # Esto construye la imagen
! $(aws ecr get-login --region $region --registry-ids $account_id --no-include-email) # Logs en AWS
! aws ecr create-repository --repository-name $ecr_repository # Crea el repositorio de ECR
! docker tag {ecr_repository + tag} $processing_repository_uri # Etiqueta la imagen para diferenciarla de otras imágenes
! docker push $processing_repository_uri # Pushea la imagen a ECR

Sending build context to Docker daemon  43.01kB
Step 1/5 : FROM python:3.10-slim-buster
 ---> 9ab67b0788d8
Step 2/5 : COPY requirements.txt .
 ---> Using cache
 ---> 042df8c4a5a4
Step 3/5 : RUN pip3 install -r requirements.txt
 ---> Using cache
 ---> 00514f171c88
Step 4/5 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 46d3d1245951
Step 5/5 : ENTRYPOINT ["python3"]
 ---> Using cache
 ---> 88a3fc3955f6
Successfully built 88a3fc3955f6
Successfully tagged preprocessing-container:latest
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:948014026119:repository/preprocessing-container",
        "registryId": "948014026119",
        "repositoryName": "preproce

1fb71801: Pushing  2.403GB/2.955GB2KPushing  349.1MB/2.955GB

1fb71801: Pushed   2.977GB/2.955GBlatest: digest: sha256:4a12540966f2f9571d13f8b5e0d8d852277465dbe8a47de8721f932b0596ea49 size: 1791


In [9]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
role = get_execution_role()
script_processor = ScriptProcessor(command=['python3'],
                                   image_uri=processing_repository_uri,
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.m5.xlarge')

In [12]:
input_data = 's3://caristo-demo/data/preprocessing/'
script_processor.run(code='radiomics.py',
                     inputs=[ProcessingInput(source=input_data, destination='/opt/ml/processing/input/data/preprocessing/')],
                     outputs=[ProcessingOutput(source='/opt/ml/processing/train', destination='s3://caristo-demo/output/preprocessing/')])

INFO:sagemaker:Creating processing-job with name preprocessing-container-2023-03-10-03-26-21-220


........................../opt/ml/processing/input/code/radiomics.py:6: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import zoom
2023-03-10 03:30:33.979418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 03:30:34.116821: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 03:30:34.116860: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up o